<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dataset-Construction" data-toc-modified-id="Dataset-Construction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset Construction</a></span><ul class="toc-item"><li><span><a href="#Load-amino-acid-sequence-and-its-solubility-(soluble-or-insoluble)-from-DeepSol" data-toc-modified-id="Load-amino-acid-sequence-and-its-solubility-(soluble-or-insoluble)-from-DeepSol-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load amino acid sequence and its solubility (soluble or insoluble) from DeepSol</a></span></li><li><span><a href="#BLAST-in-RCSB-PDB-database" data-toc-modified-id="BLAST-in-RCSB-PDB-database-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>BLAST in RCSB-PDB database</a></span></li><li><span><a href="#Blast-with-'blastp'" data-toc-modified-id="Blast-with-'blastp'-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Blast with 'blastp'</a></span></li><li><span><a href="#Blast-Sequence_id-to-GeneID" data-toc-modified-id="Blast-Sequence_id-to-GeneID-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Blast Sequence_id to GeneID</a></span></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import requests
import re
from Bio.Blast import NCBIWWW
from ssbio.pipeline.gempro import GEMPRO
from os import path
from src.utils import *
from tqdm.notebook import tqdm
import sys
from time import sleep

# Printing multiple outputs per cell
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


## Dataset Construction
### Load amino acid sequence and its solubility (soluble or insoluble) from DeepSol


In [ ]:
dir = './data/deepsol/'
aa_seq = np.concatenate((np.loadtxt(dir+'train_src', dtype='str'), 
                        np.loadtxt(dir+'test_src', dtype='str'), 
                        np.loadtxt(dir+'val_src', dtype='str')), axis=0)
sol = np.concatenate((np.loadtxt(dir+'train_tgt', dtype=int),
                      np.loadtxt(dir+'test_tgt', dtype=int),
                      np.loadtxt(dir+'val_tgt', dtype=int)), axis=0)
assert aa_seq.shape[0] == sol.shape[0]
print('There are {:d} samples in the DeepSol datasets.'.format(aa_seq.shape[0]))

l = []
for i in aa_seq:
    l.append(len(i))
print('The length of amino acid is ranged from {} to {}.'.format(min(l), max(l)))


### BLAST in RCSB-PDB database


In [ ]:
# ##################### logging #####################
# import sys

# orig_stdout = sys.stdout
# f = open('out.txt', 'w')
# sys.stdout = f

In [ ]:
# ##################### setting #####################
evalue=0.0001
# evalue = float('1E-20')

outdir_pdb = './data/pdb'

# ################### place holder ##################
# aa_seq | pdb_idx | solubility | matched pdb | if_reviewed
data = np.chararray(shape=[aa_seq.shape[0], 5], itemsize=1700)
# , unicode=True


In [ ]:
# ######## query and get matched pdb indices ########
with tqdm(total=aa_seq.shape[0], file=sys.stdout) as pbar:
    
    for idx in range(aa_seq.shape[0]):
        seq = aa_seq[idx]
    
        # write to dataset table
        # aa_seq | pdb_idx | solubility | matched pdb | if_reviewed
        data[idx, 0] = seq
        data[idx, 1] = sol[idx]

        # query on pdb-blast
        page = 'http://www.rcsb.org/pdb/rest/getBlastPDB1?' \
               'sequence={}&' \
               'eCutOff={}&' \
               'maskLowComplexity=yes&' \
               'matrix=BLOSUM62&' \
               'outputFormat=HTML'.format(seq, evalue)
        outfile = path.join('./data/pdb-blast', str(idx)+'.txt')
        response = download_from_webpage(page, outfile, idx)
        if response:
            response = re.split(r'name', response)

    
            # retrieve fully matched protein structure indices which have 100% identity
            matched_pbds = []
            if response.__len__() == 1:
                error('no retrieved pdb file - {}'.format(idx))
            else:
#                 print(response.__len__())
                for i in range(1, response.__len__()):
                    try:                                # parsing blast result
                        st = re.search('Identities = ', response[i]).span()[1]
                        ed = re.search(', Positives', response[i]).span()[0]
                        if re.search('100', response[i][st: ed]):
                            lo = re.search('<\/a>', response[i]).span()[1]
                            matched_pbds.append(response[i][lo:(lo+4)])
                    except :
                        error('paring response false - {}'.format(idx))
                
    
                # download matched pdb files and parse the resolution
                pdb_files = []
                if matched_pbds:
                    for pdb in matched_pbds:
                        page = 'http://files.rcsb.org/view/{}.pdb'.format(pdb)
                        req = requests.get(page)
                        if req.status_code == 200:
                            response = req.text
                            outfile = path.join(outdir_pdb, str(pdb)+'.pdb')
                            if outfile:
                                with open(outfile, 'w') as f:
                                    f.write(response)
                                pdb_files.append(pdb)
                            else:
                                error('can not write to file - idx:{}, pdb:{}'.format(pdb, idx))
                                continue
                        else:
                            error('web page no response for pdb download - {}'.format(idx))
                            continue
 
                    if pdb_files:
                        reso = get_resolutions(pdb_files, outdir_pdb)
                        if reso:
                            tmp = np.array(reso).astype(float)
                            i_max = np.argmax(np.nan_to_num(tmp))
#         print(pdb_files[i_max], reso[i_max])
                            data[idx, 2] = pdb_files[i_max]             # write to dataset table
                            data[idx, 3] = str(list(zip(pdb_files, reso)))
                        print(idx)
        
        
        pbar.set_description('processed: %d' % (1 + idx))
        pbar.update(1)
        sleep(1)

# close logging
# sys.stdout = orig_stdout
# f.close()

In [ ]:
data_table = pd.DataFrame(data=data, columns=['aa_seq', 'solubility', 'pdb_idx', 'matched pdb', 'if_reviewed']) 
data_table

In [ ]:

seq = 'GSHMSLFDFFKNKGSAATATDRLKLILAKERTLNLPYMEEMRKEIIAVIQKYTKSSDIHFKTLDSNQSVETIEVEIILPR'

page = 'http://www.rcsb.org/pdb/rest/getBlastPDB1?' \
       'sequence={}&' \
       'eCutOff={}&' \
       'maskLowComplexity=yes&' \
       'matrix=BLOSUM62&' \
       'outputFormat=HTML'.format(seq, evalue)

outfile = path.join('./data/pdb-blast', str(idx)+'.txt')
response = download_from_webpage(page, outfile)
response = re.split(r'name', response)


In [ ]:
matched_pbds = [] # fully matched protein structure indices which have 100% identity
if response.__len__() == 1:
    error('no retrieved pdb file')
else:
    for i in range(1, response.__len__()):
        try:# parsing blast result
            st = re.search('Identities = ', response[i]).span()[1]
            ed = re.search(', Positives', response[i]).span()[0]
            if re.search('100', response[i][st: ed]):
                lo = re.search('<\/a>', response[i]).span()[1]
                matched_pbds.append(response[i][lo:(lo+4)])
        except :
            error('paring response false - {}'.format(idx))


In [ ]:
pdb_files = []
if matched_pbds:
    for pdb in matched_pbds:
        page = 'http://files.rcsb.org/view/{}.pdb'.format(pdb)
        req = requests.get(page)
        if req.status_code == 200:
            response = req.text
            outfile = path.join(outdir_pdb, str(pdb)+'.pdb')
            if outfile:
                with open(outfile, 'w') as f:
                    f.write(response)
                pdb_files.append(pdb)
            else:
                error('can not write to file - idx:{}, pdb:{}'.format(pdb, idx))
        else:
            error('web page no response')


        


# get the pdb index which has the best resolution


In [ ]:
# aa_seq | pdb_idx | solubility | matched pdb | if_reviewed
data = np.chararray(shape=[aa_seq.shape[0], 5], itemsize=1700)
# , unicode=True
if pdb_files:
    reso = get_resolutions(pdb_files, outdir_pdb)
    i_max = np.argmax(reso)
    print(pdb_files[i_max], reso[i_max])
    data[idx, 2] = pdb_files[i_max]             # write to dataset table
    data[idx, 3] = str(list(zip(pdb_files, reso)))
# write to dataset table
# aa_seq | pdb_idx | solubility | matched pdb | if_reviewed
data[idx, 0] = aa_seq[idx]
data[idx, 1] = sol[idx]

data_table = pd.DataFrame(data=data, 
             columns=['aa_seq', 'solubility', 'pdb_idx', 'matched pdb', 'if_reviewed']) 
data_table




### Blast with 'blastp'


In [ ]:
i = 0
result_handle = NCBIWWW.qblast("blastp", "nr", aa_seq[i])
tmp = result_handle.read()                                     # query result -> string
with open('./blastp_result/{:d}.txt'.format(i), 'w') as f:
    f.write(tmp)                                               # write query results to file
rc_st = re.compile(r'\<Hit\_id\>')
rc_ed = re.compile(r'\|\<\/Hit\_id\>')
sequence_id = tmp[rc_st.search(tmp).span()[1]: rc_ed.search(tmp).span()[0]]
sequence_id = sequence_id.split('|')[-1]


### Blast Sequence_id to GeneID


In [ ]:
a = requests.get('https://www.ncbi.nlm.nih.gov/protein/XP_001351225.1')


with open('./blastp_result/{:d}.txt'.format(i), 'r') as f:
    f.write(a.content)
    
# use ncbi_uid
b = requests.get('https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id=124504967')
with open('./bbb.txt', 'wb') as f:
    f.write(a.content)
    
# use GeneID